### **Manipuler des images avec Python: partie 2**

## Introduction à l'analyse des images - 32M7132

*Adrien Jeanrenaud (adrien.jeanrenaud@unige.ch)*

<div class="alert alert-block alert-info">
<b>Manipulations des images partie 2</b> : continuons la manipulation d'images en quantité, en extrayant automatiquement ou manuellement les informations des images :
</div>

## **Plan du cours**

> **Extraire des inforamtions sur les couleurs**
> * Cibler un dossier 
> * Reprendre notre fonction pour importer des images
> * Insérer l'extraction des couleurs dominantes
> * Récupérer les données
> * Enregistrer les résultats

> **Extraire des informations manuellement**
> * Définir les variables de base
> * Que cherche-t-on ?
> * Créer un tableur
> * Reprendre un tableur en route
> * Ajouter nos informations manuellement


In [ ]:
# importer les librairies nécessaires

import numpy as np
import os
import cv2
import matplotlib.pyplot as plt

## **Extraire des inforamtions sur les couleurs**

Il y a quelques semaines, nous avons extrait les couleurs dominantes à l'aide d'un algorithme de k-moyenne (voir cours 6). Désormais, nous allons automatiser l'extraction de ces informations en insérant l'algorithme dans une fonction qui l'applique à toutes les images.

### Cibler un dossier 

Premièrement il faut, comme d'habitude, cibler le dossier avec nos images.

In [ ]:
# dans quel dossier nous sommes

os.getcwd()

In [ ]:
# Cibler un dossier

path = "fauvisme1904_1908"

In [ ]:
# est-ce bien un dossier ?

os.path.isdir(path)

In [ ]:
# fichiers et dossiers au bout du chemin

os.listdir(path)

### Reprendre notre fonction pour importer des images

La fonction que nous avons mis au point la semaine dernière est toujours d'actualité 

In [ ]:
# ligne par ligne

def importer_images(path):
    
    liste_images = []
    ext = [".jpeg", ".png", ".jpg"]
    dossier = os.listdir(path)
    for files in dossier:
        if files.endswith(tuple(ext)):
            image_dossier = os.path.join(path, files)
            image = cv2.imread(image_dossier)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            liste_images.append(image)
    return liste_images

In [ ]:
# Utilisons la fonction

mes_images = importer_images(path=path)

In [ ]:
len(mes_images)

### Insérer l'extraction des couleurs dominantes

Il est possible d'ajouter directement dans la fonction l'extraction des couleurs dominantes: pour tout faire en une fois

In [ ]:
# import la librairie
from sklearn.cluster import KMeans

In [ ]:
# Un petit test d'après son image couleur

plt.imshow(mes_images[0])
plt.show()

In [ ]:
# d'abord il faut passer d'une image 3D à une image 2D
(h,w,c) = mes_images[0].shape
img2D = mes_images[0].reshape(h*w,c)
print(img2D)
print(img2D.shape)

In [ ]:
# On choisit le nombre de regroupements (clusters) à l'aide du k

n = 6
kmeans_model = KMeans(n_clusters=n)

In [ ]:
# On applique l'algorithme à notre image

s=kmeans_model.fit(img2D)
cluster_labels = kmeans_model.fit_predict(img2D)

In [ ]:
# On peut voir combien de pixels il y a par regroupemets

from collections import Counter
labels_count = Counter(cluster_labels)
print(labels_count)

In [ ]:
# On peut voir quels sont les centres des regroupements, donc les couleurs

rgb_cols = kmeans_model.cluster_centers_.round(0).astype(int)

print(rgb_cols)

In [ ]:
# L'image est désormais faite de "k" couleurs
img_quant = np.reshape(rgb_cols[cluster_labels],(h,w,c))

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(16,12))
ax[0].imshow(mes_images[0])
ax[0].set_title('Image en couleur')
ax[1].imshow(img_quant)
ax[1].set_title(f'Image avec un k-means = {n}')

In [ ]:
img_quant.shape

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
# créer une fonction avec tout dedans et récupérer les informations

ext = [".jpeg", ".png", ".jpg"]
dossier = os.listdir(path)
for files in tqdm(dossier[:3]):
    if files.endswith(tuple(ext)):
        image_dossier = os.path.join(path, files)
        file, ext = os.path.splitext(files)
        
        image = cv2.imread(image_dossier)        
        
        (h,w,c) = image.shape
        img2D = image.reshape(h*w,c)
        
        n = 6
        kmeans_model = KMeans(n_clusters=n)
        
        s=kmeans_model.fit(img2D)
        cluster_labels = kmeans_model.fit_predict(img2D)
        
        rgb_cols = kmeans_model.cluster_centers_.round(0).astype(int)

        img_quant = np.reshape(rgb_cols[cluster_labels],(h,w,c))
        
        cv2.imwrite(f'kmeans{n}_{file}{ext}', img_quant)

<div class="alert alert-block alert-warning">
<b>Exercice</b>: reprenez simplement le code et faites en une fonction qui prend comme arguments: un chemin et le k (nombre de regroupements).</div>

## **Extraire des informations manuellement**

Que ce soit la détection de visages, l'extraction de couleur ou le traitement des métadonnées, nous avons vu comment extraire automatiquement des informations des images. Il est également possible d'extraire des informations manuellement, à l'aide d'une fonction qui automatise la tâche

### Définir les variables de base

En premier lieu, il faut définir quelques variables

> * le dossier d'images
> * le nom du csv et les extensions des images

In [ ]:
#importer les librairies

from PIL import Image
from time import sleep
plt.rcParams['figure.figsize'] = [12, 7]

In [ ]:
# le dossier à explorer
folder_to_roam = 'fauvisme1904_1908/'

os.path.isdir(folder_to_roam)

In [ ]:
# le nom du csv à écrire 
csv_to_write = 'fauve_manuel.csv'

# pour n'avoir que les images
valid_extensions = ['.jpg', '.png', '.jpeg']

# separateur dans le csv
separator = ','

### Que cherche-t-on ?

Quelle(s) question(s) pose-t-on à notre groupe d'image ?

In [ ]:
# l'information à noter
field_to_write1 = 'personne'

# la seconde information à noter
#field_to_write2 = ' '

### Créer un tableur

Il nous faut créer un tableur vide pour enregistrer nos informations. On utilise les variables pour définir les colones

In [ ]:
index = [] 
#keys = [str(field_to_write1), str(field_to_write2)] #si plusieurs informations
keys = [str(field_to_write1)]
iterator = list(os.walk(folder_to_roam))
for root, dirs, files in iterator:
    iterator_bis = files
    for file in  iterator_bis:
        if file.endswith(tuple(valid_extensions)):
            index.append(file)

pd.DataFrame(columns=keys, index=index).to_csv(csv_to_write)

In [ ]:
# voir notre csv

csv = pd.read_csv(csv_to_write, index_col = [0] )
csv

### Reprendre un tableur en route

On peut se retrouver devant le cas de figure où on reprend notre CSV après avoir déjà extrait quelques informations. Si vous ajoutez des images à votre dossier ou si vous voulez extraire d'autres informations, il faudra recommencer.

In [ ]:
todo = []
toa = []
iterator = list(os.walk(folder_to_roam))
df = pd.read_csv(csv_to_write, index_col = [0] )
for root, dirs, files in iterator:
    for file in files:
        if file.endswith(tuple(valid_extensions)):
            for i in df.index:
                if i == file:
                    a = np.where(df.index == i)[0][0]
                    if type(df.iloc[int(a)][0]) != str:
                        todo.append(file)
                        toa.append(a)
                    else:
                        continue
    print("Entrées à faire : " , len(todo))

### Ajouter nos informations manuellement

Maintenant on peut extraire les informations. La boucle vous demande toute les 10 images si vous voulez continuer. En disant non, vous vous engagez à recommencer à la cellule du dessus.

In [ ]:
df = pd.read_csv(csv_to_write, index_col = [0] )
count = 0
for j, k in zip(todo, toa):
    img = Image.open(folder_to_roam+j)
    plt.imshow(img)
    plt.show()
    sleep(0.1)
    print("Image name : ", j)
    champ1 = input(str(field_to_write1) + " : ")
    df.loc[df.index[int(k)], field_to_write1] = champ1
    #champ2 = input(str(field_to_write2) + " : ")
    #df.loc[df.index[int(k)], field_to_write2] = champ2
    df.to_csv(csv_to_write)
    count += 1
    if count % 10 == 0 :
        content = input('Continue ? [y]/n')
        if content == 'n':
            break

<div class="alert alert-block alert-warning">
<b>Exercice</b>: repartissez vous des questions à poser à ces images. Créez un csv avec le démarche ci-dessus et proposez une visualisation des résultats.</div>